In [1]:
import pandas as pd 
import numpy as np
import my_func as mf
import chardet
from datetime import datetime
import glob
import datetime
from datetime import timedelta
from datetime import datetime
import os
from datetime import date
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import re
import telegram
from telegram.ext import Updater
import asyncio
from telegram import Bot
import aiohttp
from aiofiles import open as aiofiles_open
import base64
import json
from aiohttp import ClientSession, ClientTimeout
import logging

In [2]:
#pip install --upgrade --no-deps python-telegram-bot


In [3]:
#!pip install --upgrade httpx


In [4]:
token = "your key here"
encoded_token = base64.b64encode(token.encode('utf-8')).decode('utf-8')
print(encoded_token)

YmI5ODc2OGItYWE5Mi00ZjcwLWI3MjQtMDkzNmViNmQ2NDhi


In [5]:
# Токен бота
bot_token = "your key here"
bot = Bot(token=bot_token)

# ID чата, куда будут отправляться файлы 
chat_ids = ["your key here"]
#, -4005852353


In [6]:
#!pip install python-telegram-bot
#!pip  install --upgrade python-telegram-bot

In [7]:
#!pip install aiohttp

In [8]:
# Определение директории, где хранятся папки с датами
base_dir = 'your way here'
# Получение списка всех файлов и папок в директории
all_files = os.listdir(base_dir)

In [9]:
# Фильтрация списка, чтобы оставить только папки с датами в нужном формате
date_folders = [f for f in all_files if re.match(r'\d{2}_\d{2}', f)]

In [10]:
current_date = datetime.now()

# Добавляем один день, чтобы получить завтрашнюю дату
tomorrow_date = current_date + timedelta(days=1)

# Форматируем дату в формат "дд_мм"
formatted_tomorrow_date = tomorrow_date.strftime("%d_%m")

print(formatted_tomorrow_date)

06_09


In [11]:
current_date = date.today().strftime("%d_%m")

In [12]:
current_date

'05_09'

In [13]:
# Настройка логгера
logger = logging.getLogger('send_data_to_server')
logger.setLevel(logging.DEBUG)  # Установка уровня логирования
file_handler = logging.FileHandler(
    f'C:\\Users\\maksa\\Desktop\\123finance\\заначка\\база ильи\\log\\send_data_to_server_{current_date}.log', encoding='utf-8')
file_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [14]:
# Преобразование названий папок в объекты datetime
dates = []
for folder in date_folders:
    try:
        dates.append(datetime.strptime(folder, "%d_%m"))
    except ValueError:
        continue  # Пропускаем названия, которые не соответствуют формату даты

In [15]:
def upload_rest_data(data_name, dates, base_dir):
    # Получение текущей даты в нужном формате
    current_date = date.today().strftime("%d_%m")

    # Преобразование текущей даты в объект datetime для сравнения
    current_date_dt = datetime.strptime(current_date, "%d_%m")

    # Удаление текущей даты из списка dates
    dates = [d for d in dates if d != current_date_dt]

    if dates:
        latest_date = max(dates)
        latest_date_str = latest_date.strftime("%d_%m")

        # Формирование пути к файлу с использованием последней даты
        df_staraya_path = os.path.join(
            base_dir, latest_date_str, 'остатки баз', f'rest_{data_name}_{latest_date_str}.csv')

        # Загрузка файла в DataFrame
        data_frame = pd.read_csv(df_staraya_path, low_memory=False)
        return data_frame
    else:
        print("Не найдены подходящие папки с данными.")
        return None

In [16]:
def google_sheets_service():
    creds = Credentials.from_authorized_user_file(r'C:\Users\maksa\all_repo\credentials.json')
    service = build('sheets', 'v4', credentials=creds)
    return service.spreadsheets()

In [17]:
def upload_to_google_sheets(data, spreadsheet_id, range_name):
    # Убедитесь, что data является списком списков
    service = google_sheets_service()
    request = service.values().update(spreadsheetId=spreadsheet_id, range=range_name,
                                      valueInputOption='RAW', body={'values': data})
    response = request.execute()
    print(response)


In [18]:
fincrm_staraya = upload_rest_data('fincrm', dates, base_dir)

In [19]:
#fincrm_staraya.drop('Unnamed: 0', axis=1, inplace=True)

In [20]:
fincrm_staraya.sample(2)

,phone,first_name,last_name,middle_name,region,city,unified_name
1918,79141544484,шафа,абдиева,адиловна,хабаровский край,хабаровск г,хабаровский край
1374,79527859627,людмила,лихачева,николаевна,нижегородская область,кстово,нижегородская область


In [21]:
ar_base = upload_rest_data('artur', dates, base_dir)

In [22]:
ar_base.sample(2)

,phone,first_name,last_name,middle_name,unified_name,city
308983,79817416582,сергей,павлов,владимирович,г.санкт-петербург и ленинградская область,санкт-петербург
161775,79211325208,елена,яковлева,николаевна,вологодская область,вологда


In [23]:
basis_staraya = upload_rest_data('basis', dates, base_dir)

In [24]:
basis_staraya.sample(2)

,phone,first_name,last_name,middle_name,region,city,unified_name
128937,79501987695,сергей,новосёлов,кимович,свердловская область,екатеринбург,свердловская область
145406,79186663491,владимир,костюченко,николаевич,краснодарский край,краснодар,краснодарский край


In [25]:
retro_base = upload_rest_data('retro', dates, base_dir)

In [26]:
#retro_base.drop('Unnamed: 0', axis=1, inplace=True)

In [27]:
retro_base.sample(2)

,phone,first_name,last_name,middle_name,unified_name,city,region
490533,79235441214,сай-суу,ондар,юрьевна,республика тыва,сарыг-сеп,республика тыва
322145,79309572244,александр,красильников,сергеевич,оренбургская область,оренбург,оренбургская область


In [28]:
fincrm_nofio_base = upload_rest_data('fincrm_no_fio', dates, base_dir)

In [29]:
fincrm_nofio_base.unified_name.value_counts ()

unified_name
саратовская область                         5780
ханты-мансийский автономный округ — югра    4889
волгоградская область                         27
Name: count, dtype: int64

In [30]:
iliay_base = upload_rest_data('iliay_base', dates, base_dir)

In [31]:
iliay_base.sample(2)

,phone,first_name,last_name,middle_name,region,city,unified_name
95737,79660732828,not provided,not provided,not provided,г. москва и московская область,NaN,г.москва и московская область
50273,79309595969,not provided,not provided,not provided,г. москва и московская область,NaN,г.москва и московская область


In [32]:
cript_vov_base = upload_rest_data('cript_vov', dates, base_dir)

In [33]:
cript_vov_base.sample(2)

,phone,first_name,last_name,middle_name,region,city,unified_name
109168,79245957444,not provided,not provided,not provided,республика дагестан,NaN,республика дагестан
74287,79177542390,not provided,not provided,not provided,республика башкортостан,NaN,республика башкортостан


In [34]:
danila_bfl_rob_base = upload_rest_data('danila_bfl_rob', dates, base_dir)

In [35]:
danila_bfl_rob_base.sample() 

,phone,last_name,first_name,middle_name,region,city,unified_name
169275,79312840274,петров,максим,андреевич,свердловская обл,г артемовский,свердловская область


In [36]:
evgen_base = upload_rest_data('evgen', dates, base_dir)

In [37]:
evgen_base.sample(2) 

,phone,first_name,last_name,middle_name,region,city,unified_name
104519,79233422339,дмитрий,карташов,геннадьевич,красноярский край,красноярск,красноярский край
138787,79118370737,константин,бежанский,вадимович,санкт-петербург,санкт-петербург,г.санкт-петербург и ленинградская область


In [38]:
serg_base = upload_rest_data('serg_rob', dates, base_dir)

In [39]:
serg_base.sample(2) 

,phone,first_name,last_name,middle_name,region,city,unified_name
14820,79539543730,дмитрий,not provided,not provided,ульяновская область,NaN,ульяновская область
32372,79046004212,ольга,not provided,not provided,г.санкт-петербург и ленинградская область,NaN,г.санкт-петербург и ленинградская область


In [40]:
pavel_base = upload_rest_data('pavel', dates, base_dir)

In [41]:
pavel_base.sample (2)

,phone,first_name,last_name,middle_name,region,city,unified_name
8261,79641531835,валида,гусейнова,гасан,костромская область,г кострома,костромская область
13709,79241538373,олег,карлов,анатольевич,хабаровский край,г хабаровск,хабаровский край


In [42]:
roma_base = upload_rest_data('roma', dates, base_dir)

In [43]:
roma_base.sample (2)

,phone,first_name,last_name,middle_name,region,city,unified_name
27,79888759797,not provided,not provided,not provided,республика северная осетия,NaN,республика северная осетия
201,79188349661,not provided,not provided,not provided,республика северная осетия,NaN,республика северная осетия


In [44]:
paket_kripta_rob_base = upload_rest_data('paket_kripta_rob', dates, base_dir)

In [45]:
paket_kripta_rob_base.sample (2)

,phone,first_name,last_name,middle_name,region,city,unified_name
34106,79224231439,not provided,not provided,not provided,ханты-мансийский автономный округ - югра,NaN,ханты-мансийский автономный округ — югра
43703,79091718519,not provided,not provided,not provided,ханты-мансийский автономный округ - югра,NaN,ханты-мансийский автономный округ — югра


In [46]:
traf_call_base = upload_rest_data('traf_call', dates, base_dir)

In [47]:
traf_call_base.sample (2)

,phone,last_name,first_name,middle_name,region,city,unified_name
244823,79953577794,рябова,наталья,петровна,нижегородская область,нижний новгород,нижегородская область
930225,79875032681,шалавин,павел,викторович,пензенская область,пенза,пензенская область


In [48]:
vlad_base = upload_rest_data('vlad', dates, base_dir)

In [49]:
vlad_base.head()

,phone,last_name,first_name,middle_name,region,city,unified_name


In [50]:
kirill_base = upload_rest_data('kirill', dates, base_dir)

In [51]:
kirill_base.sample (2)

,phone,first_name,last_name,middle_name,region,city,unified_name
4425,79043806569,not provided,not provided,not provided,свердловская область,NaN,свердловская область
3019,79022772935,not provided,not provided,not provided,свердловская область,NaN,свердловская область


In [52]:
fincrm=fincrm_staraya.copy ()
artur=ar_base.copy()
basis=basis_staraya.copy()
retro=retro_base.copy()
fincrm_nofio=fincrm_nofio_base.copy()
iliay=iliay_base.copy ()
cript_vov=cript_vov_base.copy()
danila_bfl_rob=danila_bfl_rob_base.copy()
evgen=evgen_base.copy()
serg_rob=serg_base.copy()
pavel=pavel_base.copy()
roma=roma_base.copy()
paket_kripta_rob=paket_kripta_rob_base.copy()
traf_call=traf_call_base.copy()
vlad=vlad_base.copy()
kirill=kirill_base.copy()

In [53]:
fincrm.drop_duplicates(subset = 'phone', inplace = True)
artur.drop_duplicates(subset = 'phone', inplace = True)
basis.drop_duplicates(subset = 'phone', inplace = True)
retro.drop_duplicates(subset = 'phone', inplace = True)
fincrm_nofio.drop_duplicates(subset = 'phone', inplace = True)
iliay.drop_duplicates(subset = 'phone', inplace = True)
cript_vov.drop_duplicates(subset = 'phone', inplace = True)
danila_bfl_rob.drop_duplicates(subset = 'phone', inplace = True)
evgen.drop_duplicates(subset = 'phone', inplace = True)
serg_rob.drop_duplicates(subset = 'phone', inplace = True)
pavel.drop_duplicates(subset = 'phone', inplace = True)
roma.drop_duplicates(subset = 'phone', inplace = True)
paket_kripta_rob.drop_duplicates(subset = 'phone', inplace = True)
traf_call.drop_duplicates(subset = 'phone', inplace = True)
vlad.drop_duplicates(subset = 'phone', inplace = True)
kirill.drop_duplicates(subset = 'phone', inplace = True)

In [54]:
filtered_dataframes = {
    'fincrm': fincrm,
    'artur': artur,
    'basis': basis,
    'retro': retro,
    'fincrm_nofio': fincrm_nofio,
    'iliay':iliay,
    'cript_vov':cript_vov,
    'danila_bfl_rob':danila_bfl_rob,
    'evgen':evgen,
    'serg_rob':serg_rob,
    'paket_kripta_rob':paket_kripta_rob,
    'pavel':pavel,
    'roma':roma,
    'traf_call':traf_call,
    'vlad':vlad,
    'kirill':kirill
       
}
#'paket_kripta_rob':paket_kripta_rob,

In [55]:
zakaz = pd.read_csv(f"C:\\Users\\{current_date}\\заказ_нарезка\\all_orders_new.csv",
                    sep=';', encoding='Windows-1251')

In [56]:
sprav = pd.read_excel(r'C:\Справочник регионов2.xlsx')

In [57]:
zakaz = mf.data_preprocessing(zakaz)

In [58]:
zakaz.loc[zakaz.region == 'ханты-мансийский автономный округ — югра', 'region'] = 'хмао'

In [59]:
zakaz

,region,source,rows,limit
0,владимирская область,danila,7143,20
1,владимирская область,traf_call,3704,30
2,калужская область,danila,5962,31
3,новгородская область,danila,3409,30
4,пензенская область,danila,6833,41
5,пензенская область,traf_call,4286,30
6,республика мордовия,danila,2368,9
7,республика мордовия,traf_call,2128,20
8,ростовская область,danila,8333,20
9,тверская область,danila,6571,46


In [60]:
count_regions=zakaz.groupby('region')['limit'].sum()

In [61]:
df_counts = pd.DataFrame(count_regions).reset_index()

In [62]:
df_counts

,region,limit
0,владимирская область,50
1,калужская область,31
2,новгородская область,30
3,пензенская область,71
4,республика мордовия,29
5,ростовская область,20
6,тверская область,66
7,тюменская область,13
8,хмао,12
9,ярославская область,30


In [63]:
region_id = pd.read_csv(f"C:\\Users\\geo_dis.csv",
                    sep=';', encoding='Windows-1251')

In [64]:
region_id

,id,name,dadata_name,capital,timezone,created_at,updated_at
0,1,Алтайский край,Алтайский край,Барнаул,7,1634674974,1708056584
1,2,Амурская область,Амурская обл,Благовещенск,9,1634675423,1701357038
2,3,Архангельская область,Архангельская обл,Архангельск,3,1634675434,1707479684
3,4,Астраханская область,Астраханская обл,Астрахань,4,1634675815,1709558148
4,5,Белгородская область,Белгородская обл,Белгород,3,1634675815,1711090104
...,...,...,...,...,...,...,...
80,81,Чеченская Республика,Респ Чеченская,Грозный,3,1634675815,1701258665
81,82,Чувашская Республика,Чувашская Республика - Чувашия,Чебоксары,3,1634675815,1688024260
82,83,Чукотский автономный округ,Чукотский АО,Анадырь,12,1634675815,1686908580
83,84,Ямало-Ненецкий автономный округ,Ямало-Ненецкий АО,Салехард,5,1634675815,1707115165


In [65]:
region_id = mf.data_preprocessing(region_id)
region_id.sample (10)

,id,name,dadata_name,capital,timezone,created_at,updated_at
15,16,ивановская область,ивановская обл,иваново,3,1634675815,1710164604
65,66,саратовская область,саратовская обл,саратов,4,1634675815,1711879546
71,72,тверская область,тверская обл,тверь,3,1634675815,1707483032
30,31,магаданская область,магаданская обл,магадан,11,1634675815,1701356502
0,1,алтайский край,алтайский край,барнаул,7,1634674974,1708056584
64,65,самарская область,самарская обл,самара,4,1634675815,1709293313
25,26,красноярский край,красноярский край,красноярск,7,1634675815,1708088973
13,14,еврейская автономная область,еврейская аобл,биробиджан,10,1634675815,1691220574
53,54,республика коми,респ коми,сыктывкар,3,1634675815,1701357062
67,68,свердловская область,свердловская обл,екатеринбург,5,1634675815,1711879561


In [66]:
df_result = pd.merge(df_counts, region_id, left_on='region', right_on='name', how='left')

In [67]:
df_result

,region,limit,id,name,dadata_name,capital,timezone,created_at,updated_at
0,владимирская область,50,7,владимирская область,владимирская обл,владимир,3,1634675815,1710256372
1,калужская область,31,20,калужская область,калужская обл,калуга,3,1634675815,1712203832
2,новгородская область,30,36,новгородская область,новгородская обл,великий новгород,3,1634675815,1709558335
3,пензенская область,71,41,пензенская область,пензенская обл,пенза,3,1634675815,1710334278
4,республика мордовия,29,57,республика мордовия,респ мордовия,саранск,3,1634675815,1707195751
5,ростовская область,20,63,ростовская область,ростовская обл,ростов-на-дону,3,1634675815,1711879592
6,тверская область,66,72,тверская область,тверская обл,тверь,3,1634675815,1707483032
7,тюменская область,13,75,тюменская область,тюменская обл,тюмень,5,1634675815,1711879441
8,хмао,12,79,хмао,хмао,ханты-мансийск,5,1634675815,1711879350
9,ярославская область,30,85,ярославская область,ярославская обл,ярославль,3,1634902587,1712235652


In [68]:
df_zakaz_region = df_result[['region', 'limit', 'id']]

In [69]:
df_zakaz_region

,region,limit,id
0,владимирская область,50,7
1,калужская область,31,20
2,новгородская область,30,36
3,пензенская область,71,41
4,республика мордовия,29,57
5,ростовская область,20,63
6,тверская область,66,72
7,тюменская область,13,75
8,хмао,12,79
9,ярославская область,30,85


In [70]:
zakaz.rows.sum()

75256

In [71]:
datasets = {
    'fincrm': fincrm,
    'артур': artur,
    'основа': basis,
    'ретро': retro,
    'fincrm_no_fio':fincrm_nofio,
    'илья':iliay,
    'cript_vov':cript_vov,
    'danila':danila_bfl_rob,
    'evgen':evgen,
    'serg_rob':serg_rob,
    'paket_kripta_rob':paket_kripta_rob,
    'pavel':pavel,
    'roma_rob':roma,
    'traf_call':traf_call,
    'vlad':vlad,
    'kirill':kirill
}
#'paket_kripta_rob':paket_kripta_rob,

In [72]:
for source, dataset in datasets.items():
    if 'unified_name' not in dataset.columns:
        print(f"В датасете от '{source}' отсутствует столбец 'unified_name'")

In [73]:
container = pd.DataFrame()


In [74]:
region_mapping = {
    'хмао': 'ханты-мансийский автономный округ — югра'#,
    #'мск': 'москва',
    #'спб': 'санкт-петербург'
    # Добавьте сюда остальные необходимые сопоставления
}

for index, row in zakaz.iterrows():
    region = row['region']
    source = row['source']
    num_rows = row['rows']
    
    # Заменяем аббревиатуру на полное название региона
    if region in region_mapping:
        region = region_mapping[region]

    if source in datasets:
        dataset = datasets[source]
        # Используем регулярное выражение для поиска точного соответствия
        mask = dataset["unified_name"].str.contains(r'\b' + region + r'\b', na=False, regex=True)
        selected_rows = dataset[mask].head(num_rows)
        dataset.drop(selected_rows.index, inplace=True)  # Удаляем выбранные строки из исходного датасета
        selected_rows['partner'] = source + '_bfl_robot'  # Добавляем столбец с источником
        selected_rows[['birthdate','amount', 'email']] = ''  # Добавляем пустые столбцы
        container = pd.concat([container, selected_rows], ignore_index=True)

In [75]:
container.loc[(container.region == 'новосибирская область') & (container.city.isna()), 'city'] = 'новосибирск'

In [76]:
container.loc[container.unified_name == 'ханты-мансийский автономный округ — югра', 'unified_name'] = 'хмао'

In [77]:
container.unified_name.value_counts(dropna=False)

unified_name
тюменская область       13181
пензенская область      11119
владимирская область    10847
тверская область         8952
ростовская область       8333
калужская область        5931
ярославская область      4478
республика мордовия      4384
хмао                     3627
новгородская область     3409
Name: count, dtype: int64

In [78]:
replace_dict = {
    'fincrm_bfl_robot': 'fin_crm_robot',
    'артур_bfl_robot': 'artur_bfl_robot',
    'основа_bfl_robot': 'osnova_robot',
    'ретро_bfl_robot': 'retro_bfl_robot',
    'fincrm_no_fio_bfl_robot':'fincrm_no_fio_bfl_robot',
    'илья_bfl_robot':'fedor_robot',
    'cript_vov_bfl_robot':'kripta_rob',
    'danila_bfl_robot':'danila_bfl_rob',
    'evgen':'evgen',
    'serg_rob':'serg_rob',
    'paket_kripta_rob':'paket_kripta_rob',
    'pavel':'pavel',
    'roma_rob':'roma',
    'traf_call':'traf_call',
    'vlad':'vlad',
    'kirill_bfl_robot':'kiril_bfl_robot' # первое что было, второе что должно стать
}

container['partner'] = container['partner'].replace(replace_dict)

#'paket_kripta_rob':'paket_kripta_rob',

In [79]:
container.sample(10)

,phone,last_name,first_name,middle_name,region,city,unified_name,partner,birthdate,amount,email
5563,79785459131,стокальский,дмитрий,not provided,владимирская обл,г александров,владимирская область,danila_bfl_rob,,,
8211,79027905601,мороз,зинаида,ивановна,владимирская область,александров,владимирская область,traf_call_bfl_robot,,,
22707,79515145972,котельников,василий,not provided,пензенская обл,г пенза,пензенская область,danila_bfl_rob,,,
68922,79227669098,not provided,not provided,not provided,ханты-мансийский ао - югра,NaN,хмао,fincrm_no_fio_bfl_robot,,,
18387,79539053988,евстафьев,алексей,геннадьевич,новгородская обл,г великий новгород,новгородская область,danila_bfl_rob,,,
39462,79085116129,иваненко,ольга,валерьевна,ростовская обл,г ростов-на-дону,ростовская область,danila_bfl_rob,,,
25384,79530268965,алексеева,ольга,леонидовна,пензенская обл,г пенза,пензенская область,danila_bfl_rob,,,
64700,79088790312,малышкина,полина,павловна,тюменская обл,г тюмень,тюменская область,danila_bfl_rob,,,
34273,79648400093,плотников,михаил,владимирович,республика мордовия,саранск,республика мордовия,traf_call_bfl_robot,,,
54829,79923002530,морозова,анастасия,андреевна,тюменская обл,г тюмень,тюменская область,danila_bfl_rob,,,


In [80]:
container.unified_name.value_counts(dropna=False)

unified_name
тюменская область       13181
пензенская область      11119
владимирская область    10847
тверская область         8952
ростовская область       8333
калужская область        5931
ярославская область      4478
республика мордовия      4384
хмао                     3627
новгородская область     3409
Name: count, dtype: int64

In [81]:
df_zakaz_region

,region,limit,id
0,владимирская область,50,7
1,калужская область,31,20
2,новгородская область,30,36
3,пензенская область,71,41
4,республика мордовия,29,57
5,ростовская область,20,63
6,тверская область,66,72
7,тюменская область,13,75
8,хмао,12,79
9,ярославская область,30,85


In [82]:
container = container.drop('region', axis=1)
container = container.rename(columns={'unified_name': 'region'})

In [83]:
new_order = ['first_name', 'last_name', 'middle_name', 'email', 'city', 'region', 'phone', 'partner', 'birthdate', 'amount']

container = container[new_order]

In [84]:
container.partner.value_counts(dropna=False)


partner
danila_bfl_rob                51755
traf_call_bfl_robot           18153
fincrm_no_fio_bfl_robot        1852
osnova_robot                   1515
fedor_robot                     726
paket_kripta_rob_bfl_robot      260
Name: count, dtype: int64

In [85]:
container.phone.count()

74261

In [86]:
container.drop_duplicates(subset = 'phone', inplace = True)

In [87]:
container.phone.count()

69357

In [88]:
#test_container = container.head()

In [89]:
#test_container.phone

In [90]:
partners = container['partner'].unique()

In [91]:
regions = container.region.unique()

In [92]:
regions

array(['владимирская область', 'калужская область',
       'новгородская область', 'пензенская область',
       'республика мордовия', 'ростовская область', 'тверская область',
       'тюменская область', 'хмао', 'ярославская область'], dtype=object)

In [93]:
container.to_excel(
    f"C:\\Users\\{current_date}\\на отправку\\unity_base_{current_date}_API.xlsx", index=False)

In [94]:
async def send_document(chat_id, document, caption):
    try:
        await bot.send_document(chat_id=chat_id, document=document, caption=caption)
    except Exception as e:
        print(f"Ошибка при отправке документа: {e}")

In [95]:
def get_region_data(region_name):
    row = df_zakaz_region[df_zakaz_region['region'] == region_name].iloc[0]
    return {
        'region_id': row['id'],
        'limit': row['limit']
    }

In [96]:
async def send_data_to_server(region_id, limit, data, encoded_token):
    url = "https://dev.crmcalls.ru/api/base_api/apiBase/"
    # url = "https://gate.crmcalls.ru/api/base_api/apiBase/"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Basic {token}'
    }

    modified_data = []
    for entry in data:
        fields = {
            "first_name": entry.get("first_name", ""),
            "last_name": entry.get("last_name", ""),
            "middle_name": entry.get("middle_name", ""),
            "email": entry.get("email", ""),
            "city": entry.get("city", ""),
            "region": entry.get("region", ""),
            "birthdate": entry.get("birthdate", ""),
            "amount": entry.get("amount", ""),
            "partner": entry.get("partner", "")
        }
        modified_entry = {
            "phone": entry.get("phone", ""),
            "fields": fields
        }
        modified_data.append(modified_entry)

    body = {
        'region_id': region_id,
        'limit': limit,
        'extraphones': 0, # 1 дополнительная заливка, 0 - новая на завтра, если дозаливка - лимит 50+
        'base': modified_data
        
    }

    timeout = ClientTimeout(total=120)

    async with ClientSession(timeout=timeout) as session:
        try:
            logger.debug(
                f"Sending data to server with region_id: {region_id} and limit: {limit}")
            logger.debug(
                f"Data: {json.dumps(body, ensure_ascii=False, indent=2)}")
            async with session.post(url, json=body, headers=headers) as response:
                response_text = await response.text()
                if response.status != 200:
                    logger.error(
                        f"Unexpected status: {response.status}; Response: {response_text}")
                    return response.status, response_text
                logger.info(
                    f"Status: {response.status}; Response: {response_text if response_text else 'No content returned'}")
                return response.status, response_text

        except asyncio.TimeoutError as e:
            logger.error(f"Timeout error occurred: {str(e)}", exc_info=True)
            return None, f"Timeout error occurred: {str(e)}"
        except Exception as e:
            logger.error(
                f"An unexpected error occurred: {str(e)}", exc_info=True)
            return None, f"An unexpected error occurred: {str(e)}"

In [97]:
async def main(df_zakaz_region, container, encoded_token):
    for index, row in df_zakaz_region.iterrows():
        region_name = row['region']
        region_id = row['id']
        limit = row['limit']
        
        # Выбираем строки, соответствующие текущему региону и заменяем NaN
        region_data = container[container['region'] == region_name].fillna('')

        # Формируем данные для отправки
        data_to_send = region_data.to_dict('records')
        
        # Отправляем данные партнеру
        status, response = await send_data_to_server(region_id, limit, data_to_send, encoded_token)
        print(f"Sending data to server with region_id: {region_id} and limit: {limit}")
        #print(f"Data: {json.dumps(data_to_send, ensure_ascii=False, indent=2)}")
        print(f"Status: {status}; Response: {response}")
        
        #Отправка фойлов по регионам в бота (страховка для Ильи)
        for region in regions:
            region_data = container[container['region'] == region]
            file_path = f"C:\\{current_date}\\на отправку_test\\{region}_{current_date}_for_{formatted_tomorrow_date}.xlsx"
            region_data.to_excel(file_path, index=False)
            #for chat_id in chat_ids:
                #with open(file_path, 'rb') as file:
                    #await send_document(chat_id, file, f"{region}_{current_date}_for_{formatted_tomorrow_date}.xlsx")
    

In [98]:
if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    
    # Проверяем, запущен ли уже цикл событий
    if loop.is_running():
        loop.create_task(main(df_zakaz_region, container, encoded_token))
    else:
        loop.run_until_complete(main(df_zakaz_region, container, encoded_token))

In [99]:
def clear_google_sheets(spreadsheet_id, sheet_name):
    service = google_sheets_service()
    # Для очистки всего листа, используем имя листа без указания диапазона ячеек
    # Пример range_name: '{name}' вместо '{name}!A1'
    full_range_name = sheet_name  # sheet_name - это переменная с именем листа
    request = service.values().clear(spreadsheetId=spreadsheet_id, range=full_range_name, body={})
    response = request.execute()
    print(f"Cleared data from: {full_range_name}")

In [100]:
def save_data_and_counts(dataset, name, current_date):
    # Сохраняем остатки базы в формате CSV
    dataset.drop_duplicates(subset = 'phone', inplace = True)
    dataset.to_csv(
        f"C:\\Users\\{current_date}\\остатки баз\\rest_{name}_{current_date}.csv", index=False)

    # Подсчитываем количество строк по регионам и сохраняем в формате EXCEL
    region_counts = dataset['unified_name'].value_counts()
    region_counts.to_excel(
        f"C:\\Users\\{current_date}\\потенциал\\{name}_{current_date}_poten_rest.xlsx")

    # Преобразовываем данные для загрузки в Google Sheets
    region_counts_df = region_counts.reset_index()
    region_counts_df.columns = ['Region', 'Count']
    data = region_counts_df.values.tolist()

    spreadsheet_id = 'your key here'
    range_name = f'{name}!A1'  # Диапазон для вставки данных
    
    #лист для очистки
    sheet_name = name

    # Очистка данных на листе перед загрузкой новых
    clear_google_sheets(spreadsheet_id, sheet_name)

    # Загрузка новых данных
    upload_to_google_sheets(data, spreadsheet_id, range_name)

In [101]:
# Вызываем функцию для каждого источника
save_data_and_counts(fincrm, 'fincrm', current_date)
save_data_and_counts(artur, 'artur', current_date)
save_data_and_counts(basis, 'basis', current_date)
save_data_and_counts(retro, 'retro', current_date)
save_data_and_counts(fincrm_nofio, 'fincrm_no_fio', current_date)
save_data_and_counts(iliay, 'iliay_base', current_date)
save_data_and_counts(cript_vov, 'cript_vov', current_date)
save_data_and_counts(danila_bfl_rob, 'danila_bfl_rob', current_date)
save_data_and_counts(evgen, 'evgen', current_date)
save_data_and_counts(serg_rob, 'serg_rob', current_date)
save_data_and_counts(pavel, 'pavel', current_date)
save_data_and_counts(roma, 'roma', current_date)
save_data_and_counts(paket_kripta_rob, 'paket_kripta_rob', current_date)
save_data_and_counts(traf_call, 'traf_call', current_date)
save_data_and_counts(vlad, 'vlad', current_date)
save_data_and_counts(kirill, 'kirill', current_date)

Cleared data from: fincrm
{'spreadsheetId': '1xGrphQs_Y_9qZQ6FpSz_JsIj2RkWNttm_fEq4kCCzls', 'updatedRange': 'fincrm!A1:B53', 'updatedRows': 53, 'updatedColumns': 2, 'updatedCells': 106}
Cleared data from: artur
{'spreadsheetId': '1xGrphQs_Y_9qZQ6FpSz_JsIj2RkWNttm_fEq4kCCzls', 'updatedRange': 'artur!A1:B39', 'updatedRows': 39, 'updatedColumns': 2, 'updatedCells': 78}
Cleared data from: basis
{'spreadsheetId': '1xGrphQs_Y_9qZQ6FpSz_JsIj2RkWNttm_fEq4kCCzls', 'updatedRange': 'basis!A1:B55', 'updatedRows': 55, 'updatedColumns': 2, 'updatedCells': 110}
Cleared data from: retro
{'spreadsheetId': '1xGrphQs_Y_9qZQ6FpSz_JsIj2RkWNttm_fEq4kCCzls', 'updatedRange': 'retro!A1:B53', 'updatedRows': 53, 'updatedColumns': 2, 'updatedCells': 106}
Cleared data from: fincrm_no_fio
{'spreadsheetId': '1xGrphQs_Y_9qZQ6FpSz_JsIj2RkWNttm_fEq4kCCzls', 'updatedRange': 'fincrm_no_fio!A1:B3', 'updatedRows': 3, 'updatedColumns': 2, 'updatedCells': 6}
Cleared data from: iliay_base
{'spreadsheetId': '1xGrphQs_Y_9qZQ6Fp